Coordinate systems

ielu saves its electrode coordinates in Surface RAS (tkrRAS) coordinate system.
https://surfer.nmr.mgh.harvard.edu/fswiki/CoordinateSystems

ielu geometry gives some ways to accesss freesurfer affine transform linked to a subject and their resampled base MRI

One thing that is quite nice is that freesurfer includes freeview which can display multiple volumes and surfaces and their coordinates at the same time. You can even specify a 3D coordinate and have the cursor move there in one of these systems:
the CT might be specified in Column, Row, Slice (CRS) form

RAS stands for Right, Anterior, Superior and refers to the axes and direction of increase

LPI stands for left, posterior, inferior


In [27]:
from __future__ import division, print_function
import ielu.geometry
import numpy as np
from ielu.geometry import apply_affine
# We will need both a "dot" product and matrix inverse "inv"
from numpy.linalg import inv
from numpy import dot

In [5]:
# placeholder to put images of brain and coordiante system

Below we will define a point taken from freeview -- all listed while the mri brain volume is loaded and the registered ct from ct_nas.nii is loaded. Thus these coordinates all represent the same point just in different coordinate systems.
I chose a point near the surface of brain/skull in the right parietal region.

In [6]:
pt_ras = 64.94, 11.36, 65.2     # RAS 64.94, 11.36, 65.2
pt_tkreg = 57.85, 27.58, -44.66 # TkReg RAS 57.85, 27.58, -44.66
pt_rawavg = 66,176, 106 # rawavg [66,176, 106] row,col,slice? voxel
pt_brain = 70,100, 83   # brain [70,100, 83]
pt_ct = 66, 176, 106    # ct_nas.nii [66, 176, 106]
pt_surf_ras = 57.85, -44.66, 27.58   # rh.pial SurfaceRAS 57.85, -44.66, 27.58

In [7]:
# In freeview, I have loaded both an MRI and a co-registered CT ct_nas.nii.gz
ctfile = '/Users/clee/subjects/JJE2016/mri/ct_nas.nii.gz' 

ct_affine_vox2ras_tkr = ielu.geometry.get_vox2rasxfm(ctfile, stem='vox2ras-tkr')
ct_affine_vox2ras = ielu.geometry.get_vox2rasxfm(ctfile, stem='vox2ras')

In [8]:
ct_affine_vox2ras_tkr

array([[  -0.9375,    0.    ,    0.    ,  120.    ],
       [   0.    ,    0.    ,    1.    ,  -78.    ],
       [   0.    ,   -0.9375,    0.    ,  120.    ],
       [   0.    ,    0.    ,    0.    ,    1.    ]])

In [9]:
ct_affine_vox2ras

array([[  -0.9375,    0.    ,   -0.    ,  127.094 ],
       [   0.    ,   -0.9375,   -0.    ,  176.021 ],
       [   0.    ,    0.    ,    1.    ,  -40.376 ],
       [   0.    ,    0.    ,    0.    ,    1.    ]])

In [10]:
ct_ras_tkr2ras = np.dot(ct_affine_vox2ras,inv(ct_affine_vox2ras_tkr))
ct_ras_tkr2ras

array([[  1.   ,   0.   ,   0.   ,   7.094],
       [  0.   ,   0.   ,   1.   ,  56.021],
       [  0.   ,   1.   ,   0.   ,  37.624],
       [  0.   ,   0.   ,   0.   ,   1.   ]])

In [11]:
mri_brain = '/Users/clee/subjects/JJE2016/mri/brain.mgz'
mri_brain_vox2ras_tkr = ielu.geometry.get_vox2rasxfm(mri_brain, stem='vox2ras-tkr')
mri_brain_vox2ras = ielu.geometry.get_vox2rasxfm(mri_brain, stem='vox2ras')

In [12]:
mri_brain_vox2ras_tkr

array([[  -1.,    0.,    0.,  128.],
       [   0.,    0.,    1., -128.],
       [   0.,   -1.,    0.,  128.],
       [   0.,    0.,    0.,    1.]])

In [13]:
mri_brain_vox2ras

array([[  -1.     ,    0.     ,    0.     ,  135.09399],
       [   0.     ,    0.     ,    1.     ,  -71.97899],
       [   0.     ,   -1.     ,    0.     ,  165.62399],
       [   0.     ,    0.     ,    0.     ,    1.     ]])

In [14]:
def homogenize(v):
    """turn a 3D vector/sequence into its equivalnent homogenous coordinate vector
    and return the result as a numpy array"""
    return np.array((v[0], v[1],v[2], 1))

# create a shortcut h for the function
h = homogenize

def undo_homogenize(v4d):
    """4 components to 3 components"""
    return v4d[:3]
ih = undo_homogenize

In [15]:
# Now let's see if our points match up
# w = A v
w = dot(inv(ct_affine_vox2ras), homogenize(pt_ras))
w # note this is close to pt_ct

array([  66.2976,  175.6384,  105.576 ,    1.    ])

In [16]:
np.round(w) # if we round to the nearest integer we get back our voxel coordinate

array([  66.,  176.,  106.,    1.])

In [17]:
# try going from RAS -> surfaceRAS
T = dot(mri_brain_vox2ras_tkr, inv(mri_brain_vox2ras))
y = dot(T,h(pt_ras))
print('pt_ras:', pt_ras)
print('pt_surf_ras:', pt_surf_ras)
print('y:', y)
# does pt_surf_ras = y = T . pt_ras? Not exactly but they are very close
print('are they close?\n h(pt_surf_ras) -  y < 0.01 :', h(pt_surf_ras) -  y < 0.01)

pt_ras: (64.94, 11.36, 65.2)
pt_surf_ras: (57.85, -44.66, 27.58)
y: [ 57.84601 -44.66101  27.57601   1.     ]
are they close?
 h(pt_surf_ras) -  y < 0.01 : [ True  True  True  True]


In [40]:
from IPython.display import display, Math,Latex


$ {\vec w} = {\matrix A} {\vec v}$